In [1]:
import gc
import importlib
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import seaborn as sns
from atmacup_18 import constants

import utils

importlib.reload(utils)

<module 'utils' from '/home/tatsuya/projects/atmacup/atmacup_18/experiments/main/v00/v00_24_01/utils.py'>

In [2]:
RANDOM_STATE = 2024
utils.seed_everything(RANDOM_STATE)

## データ読み込み

In [3]:
notebook_dir = Path().resolve()
DATA_DIR = notebook_dir.parents[3].joinpath("data")
DATASET_DIR = DATA_DIR.joinpath("atmaCup#18_dataset")
TR_FEATURES_CSV = DATASET_DIR.joinpath("train_features.csv")
TS_FEATURES_CSV = DATASET_DIR.joinpath("test_features.csv")
IMAGES_DIR = DATASET_DIR.joinpath("images")
TRAFFIC_LIGHTS_CSV = DATASET_DIR.joinpath("traffic_lights.csv")

IMAGE_NAMES = ["image_t.png", "image_t-0.5.png", "image_t-1.0.png"]
TRAFFIC_LIGHTS_BBOX_IMAGE_NAME = constants.TRAFFIC_LIGHT_BBOX_IMAGE_NAME
DEPTH_IMAGE_FILE_PREFIX = constants.DEPTH_IMAGE_FILE_PREFIX
DEPTH_IMAGE_NAMES = [
    f"{DEPTH_IMAGE_FILE_PREFIX}{image_name}.npy" for image_name in IMAGE_NAMES
]

BASE_PRED_DIR = Path("..", "..", "..", "main2", "v00", "v00_05_00")
BASE_OOF_PRED_CSV = BASE_PRED_DIR.joinpath("oof_preds.csv")
BASE_SUBMISSION_CSV = BASE_PRED_DIR.joinpath("submission.csv")

In [4]:
TARGET_COLS = sum([[f"x_{i}", f"y_{i}", f"z_{i}"] for i in range(6)], [])
BASE_PRED_COLS = [f"base_pred_{col}" for col in TARGET_COLS]

In [5]:
tr_df = utils.read_feature_csv(TR_FEATURES_CSV)
tr_df.head(2)

ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,y_4,z_4,x_5,y_5,z_5,scene_id,scene_dsec,origin_idx
str,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32,i64
"""00066be8e20318869c38c66be46663…",5.701526,1.538456,-2.165777,-139.0,0.0,false,0.25,true,"""drive""",false,false,2.82959,0.032226,0.045187,6.231999,0.065895,0.107974,9.785009,0.124972,0.203649,13.485472,0.163448,0.302818,17.574227,0.174289,0.406331,21.951269,0.199503,0.485079,"""00066be8e20318869c38c66be46663…",320,0
"""00066be8e20318869c38c66be46663…",11.176292,0.279881,-11.625697,-44.0,0.0,false,0.0,false,"""drive""",false,true,4.970268,-0.007936,0.005028,10.350489,-0.032374,-0.020701,15.770054,0.084073,0.008645,21.132415,0.391343,0.036335,26.316489,0.843124,0.065,31.383814,1.42507,0.073083,"""00066be8e20318869c38c66be46663…",420,1


In [6]:
ts_df = utils.read_feature_csv(TS_FEATURES_CSV)
ts_df.head(2)

ID,vEgo,aEgo,steeringAngleDeg,steeringTorque,brake,brakePressed,gas,gasPressed,gearShifter,leftBlinker,rightBlinker,scene_id,scene_dsec,origin_idx
str,f64,f64,f64,f64,f64,bool,f64,bool,str,bool,bool,str,i32,i64
"""012baccc145d400c896cb82065a93d…",3.374273,-0.01936,-34.008415,17.0,0.0,false,0.0,false,"""drive""",false,false,"""012baccc145d400c896cb82065a93d…",120,0
"""012baccc145d400c896cb82065a93d…",2.441048,-0.022754,307.860077,295.0,0.0,true,0.0,false,"""drive""",false,false,"""012baccc145d400c896cb82065a93d…",220,1


In [7]:
def reduce_base_pred(
    df: pl.DataFrame, base_pred_df: pl.DataFrame, has_target: bool
) -> pl.DataFrame:
    """
    基礎推定値のdfをconcatし、元のdfのtarget列から除去したdfを返す

    Args:
        df (pl.DataFrame): target列を持つDataFrame
        base_pred_df (pl.DataFrame): 基礎推定値のDataFrame
        has_target (bool): target列を持つかどうか
    """
    target_cols = TARGET_COLS
    base_pred_cols = BASE_PRED_COLS
    df = pl.concat(
        [
            df,
            base_pred_df.select(TARGET_COLS).rename(
                {t: b for t, b in zip(target_cols, base_pred_cols)}
            ),
        ],
        how="horizontal",
    )

    if has_target:
        df = df.with_columns(
            [
                (pl.col(tg_col) - pl.col(base_pred_col)).alias(tg_col)
                for tg_col, base_pred_col in zip(target_cols, base_pred_cols)
            ]
        )
    return df


def add_base_pred_to_target(df: pl.DataFrame, target_cols: list[str]) -> pl.DataFrame:
    """
    target_colsの列に基礎推定量を足したDataFrameを返す

    Args:
        df (pl.DataFrame): target列を持つDataFrame
        target_cols (list[str]): 追加する基礎推定量の列名
    """
    df = df.with_columns(
        [
            (pl.col(tg_col) + pl.col(bp_col)).alias(tg_col)
            for tg_col, bp_col in zip(target_cols, BASE_PRED_COLS)
        ]
    )

    return df


if BASE_PRED_DIR is not None:
    # columns: "x_0", "y_0", "z_0", ..., "x_5", "y_5", "z_5"
    base_oof_pred_df = pl.read_csv(BASE_OOF_PRED_CSV)
    base_submission_df = pl.read_csv(BASE_SUBMISSION_CSV)

    # 基礎推定値を元のtarget列から引いた値を新たなtarget列とする
    tr_df = reduce_base_pred(tr_df, base_oof_pred_df, has_target=True)
    ts_df = reduce_base_pred(ts_df, base_submission_df, has_target=False)

    del base_oof_pred_df, base_submission_df
    gc.collect()

In [8]:
tr_tl_bbox_images = utils.load_npy_images(
    IMAGES_DIR,
    ids=tr_df.get_column("ID").to_list(),
    image_names=[TRAFFIC_LIGHTS_BBOX_IMAGE_NAME],
)
print(tr_tl_bbox_images.shape)
ts_tl_bbox_images = utils.load_npy_images(
    IMAGES_DIR,
    ids=ts_df.get_column("ID").to_list(),
    image_names=[TRAFFIC_LIGHTS_BBOX_IMAGE_NAME],
)
print(ts_tl_bbox_images.shape)

(43371, 1, 64, 128, 8)
(1727, 1, 64, 128, 8)


In [9]:
tr_depth_images = utils.load_npy_images(
    IMAGES_DIR,
    ids=tr_df.get_column("ID").to_list(),
    image_names=DEPTH_IMAGE_NAMES,
)
print(tr_depth_images.shape)
ts_depth_images = utils.load_npy_images(
    IMAGES_DIR,
    ids=ts_df.get_column("ID").to_list(),
    image_names=DEPTH_IMAGE_NAMES,
)
print(ts_depth_images.shape)

(43371, 3, 64, 128, 1)
(1727, 3, 64, 128, 1)


In [10]:
tr_images = utils.load_images(
    IMAGES_DIR, ids=tr_df.get_column("ID").to_list(), image_names=IMAGE_NAMES
)
print(tr_images.shape)
ts_images = utils.load_images(
    IMAGES_DIR, ids=ts_df.get_column("ID").to_list(), image_names=IMAGE_NAMES
)
print(ts_images.shape)

(43371, 3, 64, 128, 3)
(1727, 3, 64, 128, 3)


In [11]:
tr_images = utils.preprocess_images(
    # [tr_images, tr_tl_bbox_images, tr_optical_flow_images]
    [tr_images, tr_tl_bbox_images, tr_depth_images]
    # [tr_images]
)
ts_images = utils.preprocess_images(
    # [ts_images, ts_tl_bbox_images, ts_optical_flow_images]
    [ts_images, ts_tl_bbox_images, ts_depth_images]
    # [ts_images]
)

print(tr_images.shape)
print(ts_images.shape)

(43371, 20, 64, 128)
(1727, 20, 64, 128)


In [ ]:
del tr_tl_bbox_images
gc.collect()
del ts_tl_bbox_images
gc.collect()
del tr_depth_images
gc.collect()
del ts_depth_images
gc.collect()

## scene_dsec順に並び替える

In [13]:
tr_df = tr_df.sort(["scene_id", "scene_dsec"])
ts_df = ts_df.sort(["scene_id", "scene_dsec"])

tr_images = tr_images[tr_df.get_column("origin_idx").to_numpy()]
ts_images = ts_images[ts_df.get_column("origin_idx").to_numpy()]

## Target

In [14]:
target = utils.CoordinateTarget(prefix="tg_")
target.fit(tr_df)

tg_df = target.transform(tr_df)
print(tg_df.columns)
print(tg_df.describe().glimpse())
tr_df = pl.concat([tr_df, tg_df], how="horizontal")

del tg_df
gc.collect()

['tg_cood_x_0', 'tg_cood_y_0', 'tg_cood_z_0', 'tg_cood_x_1', 'tg_cood_y_1', 'tg_cood_z_1', 'tg_cood_x_2', 'tg_cood_y_2', 'tg_cood_z_2', 'tg_cood_x_3', 'tg_cood_y_3', 'tg_cood_z_3', 'tg_cood_x_4', 'tg_cood_y_4', 'tg_cood_z_4', 'tg_cood_x_5', 'tg_cood_y_5', 'tg_cood_z_5']
Rows: 9
Columns: 19
$ statistic   <str> 'count', 'null_count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'
$ tg_cood_x_0 <f64> 43371.0, 0.0, 3.915807866824304e-06, 0.09755740299606562, -2.068491360867153, -0.0399921260776388, 0.0001685152283451563, 0.041051261445355536, 1.205429121318332
$ tg_cood_y_0 <f64> 43371.0, 0.0, 3.3850710814478617e-05, 0.062100404071971656, -2.5727879141483267, -0.021748391426786483, -0.0004883458396699972, 0.021632620544553012, 3.813253350830932
$ tg_cood_z_0 <f64> 43371.0, 0.0, 1.0609752174614824e-05, 0.04015807466563177, -0.9997029502068319, -0.018513304908815843, 0.00038892404923006546, 0.017899114943309744, 1.4469071368673965
$ tg_cood_x_1 <f64> 43371.0, 0.0, -6.5520745548367315e-06, 

0

## 特徴量

In [15]:
feature = utils.Feature(prefix="ft_")
feature.fit(tr_df)

ft_df = feature.transform(tr_df)
print(ft_df.columns)
print(ft_df.describe().glimpse())
tr_df = pl.concat([tr_df, ft_df], how="horizontal")

ft_df = feature.transform(ts_df)
print(ft_df.columns)
print(ft_df.describe().glimpse())
ts_df = pl.concat([ts_df, ft_df], how="horizontal")

del ft_df
gc.collect()

['ft_vEgo', 'ft_aEgo', 'ft_steeringAngleDeg', 'ft_steeringTorque', 'ft_brake', 'ft_brakePressed', 'ft_gas', 'ft_gasPressed', 'ft_is_gearShifter_drive', 'ft_is_gearShifter_neutral', 'ft_is_gearShifter_park', 'ft_is_gearShifter_reverse', 'ft_leftBlinker', 'ft_rightBlinker', 'ft_base_pred_x0', 'ft_base_pred_y0', 'ft_base_pred_z0', 'ft_base_pred_x1', 'ft_base_pred_y1', 'ft_base_pred_z1', 'ft_base_pred_x2', 'ft_base_pred_y2', 'ft_base_pred_z2', 'ft_base_pred_x3', 'ft_base_pred_y3', 'ft_base_pred_z3', 'ft_base_pred_x4', 'ft_base_pred_y4', 'ft_base_pred_z4', 'ft_base_pred_x5', 'ft_base_pred_y5', 'ft_base_pred_z5']
Rows: 9
Columns: 33
$ statistic                 <str> 'count', 'null_count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'
$ ft_vEgo                   <f64> 43371.0, 0.0, 9.172175823216334, 7.226919878374694, -0.1619189828634262, 2.5786657333374023, 8.518790245056152, 14.286815643310547, 27.55126190185547
$ ft_aEgo                   <f64> 43371.0, 0.0, -0.015654028629347255, 0.63

0

## モデリング

In [16]:
N_SPLITS = 2

In [17]:
n_sample_in_scene = 3

model_params = {
    "dnn": {
        "n_sample_in_scene": n_sample_in_scene,
        "n_img_channels": tr_images.shape[1] * n_sample_in_scene,
        "n_features": len(feature.columns) * n_sample_in_scene,
        "n_targets": len(target.columns),
        "dropout": 0.0,
        "embed_dim": 128,
        "n_layers": 1,
    },
    "dnn_pretrained_model": {
        # list[str]: len(list) == n_splits
        "weight_path": None,
        "load_only_backbone": None,
    },
    "dev": "cuda",
}

lr = 5e-5
fit_params = {
    "dnn": {
        "tr_batch_size": 32,
        "vl_batch_size": 128,
        "trainer_params": {
            "criterion_params": {},
            "opt": "adamw",
            "opt_params": {"lr": lr, "weight_decay": 1e-4},
            "backbone_opt_params": {"lr": lr, "weight_decay": 1e-4},
            "sch_params": {
                "max_lr": lr,
                "pct_start": 0.1,
                "div_factor": 25,
                "final_div_factor": 1000,
            },
            "epochs": 10,
            "dev": "cuda",
            "val_freq": 1,
            "prefix": "",
            "save_best": False,
            "save_epochs": [],
            "maximize_score": False,
            "grad_max_norm": None,
        },
    },
}

In [18]:
models, oof_preds = utils.train(
    model_params=model_params,
    fit_params=fit_params,
    df=tr_df,
    images=tr_images,
    target_cols=target.columns,
    feature_cols=feature.columns,
    group_col="scene_id",
    scene_id_col="scene_id",
    scene_dsec_col="scene_dsec",
    n_splits=N_SPLITS,
)

-----------------
-----------------
Training fold 0...
train samples: 21685, valid samples: 21686
Save model : fold0_model.pth

epoch  0
lr  2.000000000000001e-06
lr  2.000000000000001e-06
lr  2.000000000000001e-06
lr  2.000000000000001e-06


100%|██████████| 170/170 [00:27<00:00,  6.28it/s]



Train Loss: 5.7194
{'loss': 5.719424257954607, 'loss_mse_0': 0.09824901613430223, 'loss_mse_1': 0.05995119820889553, 'loss_mse_2': 0.04146437944247328, 'loss_mse_3': 0.1974756087739147, 'loss_mse_4': 0.11553291236071062, 'loss_mse_5': 0.08552862000287108, 'loss_mse_6': 0.33715816117283864, 'loss_mse_7': 0.19134742736354185, 'loss_mse_8': 0.12804608049759836, 'loss_mse_9': 0.5180253491267848, 'loss_mse_10': 0.3075647605816934, 'loss_mse_11': 0.17380935982133824, 'loss_mse_12': 0.7562555072930927, 'loss_mse_13': 0.47951728112698483, 'loss_mse_14': 0.22079321592936135, 'loss_mse_15': 1.0326451539685138, 'loss_mse_16': 0.708247002296733, 'loss_mse_17': 0.2678132313613173}
Valid Loss: 5.6894
{'loss': 5.689441560296451, 'loss_mse_0': 0.10479758342399317, 'loss_mse_1': 0.07377367638708913, 'loss_mse_2': 0.03844657730968559, 'loss_mse_3': 0.21417238396756788, 'loss_mse_4': 0.1367355124477078, 'loss_mse_5': 0.07808447863687487, 'loss_mse_6': 0.33737624445382286, 'loss_mse_7': 0.208889645644847

100%|██████████| 170/170 [00:27<00:00,  6.23it/s]



Train Loss: 5.6238
{'loss': 5.623836965645434, 'loss_mse_0': 0.09589488446943605, 'loss_mse_1': 0.05877675044940485, 'loss_mse_2': 0.0410126495729455, 'loss_mse_3': 0.19429309678262588, 'loss_mse_4': 0.11344380184809953, 'loss_mse_5': 0.08364730124723611, 'loss_mse_6': 0.32544508937916866, 'loss_mse_7': 0.18842130638603974, 'loss_mse_8': 0.12682874712328174, 'loss_mse_9': 0.5101693434038226, 'loss_mse_10': 0.30454425383881945, 'loss_mse_11': 0.17083528320933727, 'loss_mse_12': 0.7428327125759914, 'loss_mse_13': 0.4720731997749654, 'loss_mse_14': 0.21681270672440353, 'loss_mse_15': 1.022262944912805, 'loss_mse_16': 0.6929565624745241, 'loss_mse_17': 0.2635863427089271}
Valid Loss: 5.6964
{'loss': 5.696428345231449, 'loss_mse_0': 0.10509217929314164, 'loss_mse_1': 0.07318957529085524, 'loss_mse_2': 0.038531393311260376, 'loss_mse_3': 0.21346538277233348, 'loss_mse_4': 0.13488817764993977, 'loss_mse_5': 0.07817828788915102, 'loss_mse_6': 0.3392965473672923, 'loss_mse_7': 0.20782712071257

100%|██████████| 170/170 [00:27<00:00,  6.19it/s]



Train Loss: 5.6066
{'loss': 5.606576039308635, 'loss_mse_0': 0.0956245728010419, 'loss_mse_1': 0.05874105843480132, 'loss_mse_2': 0.04094163355509719, 'loss_mse_3': 0.19380202497103893, 'loss_mse_4': 0.11329651916704996, 'loss_mse_5': 0.08333195613606377, 'loss_mse_6': 0.324839120656472, 'loss_mse_7': 0.18833153531989813, 'loss_mse_8': 0.12628173291617165, 'loss_mse_9': 0.5081603984779058, 'loss_mse_10': 0.3038627795291792, 'loss_mse_11': 0.17016158057610606, 'loss_mse_12': 0.7405621489644755, 'loss_mse_13': 0.47052086034632856, 'loss_mse_14': 0.2157937480656752, 'loss_mse_15': 1.0199749149723278, 'loss_mse_16': 0.690045396963814, 'loss_mse_17': 0.26230403649974254}
Valid Loss: 5.7372
{'loss': 5.737242035304798, 'loss_mse_0': 0.10561877432114938, 'loss_mse_1': 0.07334579574492048, 'loss_mse_2': 0.038361592662027655, 'loss_mse_3': 0.2143013398436939, 'loss_mse_4': 0.1358464578714441, 'loss_mse_5': 0.07784279493943733, 'loss_mse_6': 0.3400127649307251, 'loss_mse_7': 0.20990599687485134,

 15%|█▌        | 104/677 [00:14<01:17,  7.37it/s]


KeyboardInterrupt: 

In [ ]:
oof_preds = oof_preds.select(pl.all().name.prefix("pred_"))
pred_cols = oof_preds.columns

tr_df = pl.concat([tr_df, oof_preds], how="horizontal")
tr_df

## 評価

In [ ]:
def calc_score(df: pl.DataFrame, pred_cols: list[str]):
    tg_cols = sum([[f"x_{i}", f"y_{i}", f"z_{i}"] for i in range(6)], [])

    tg = df.select(tg_cols).to_numpy()
    pred = df.select(pred_cols).to_numpy()

    scores = np.abs(tg - pred).mean(axis=0)
    scores = {f"score_{col}": float(score) for col, score in zip(pred_cols, scores)}
    scores["avg"] = float(np.abs(tg - pred).mean())
    return scores


scores = calc_score(tr_df, pred_cols)
scores

In [ ]:
utils.plot_calibration_curve(tr_df, pred_cols, n_bins=40)

In [ ]:
if BASE_PRED_DIR is not None:
    # 差し引いていた基礎推定値を足して元のtarget, pred列に戻す
    tr_df = add_base_pred_to_target(tr_df, TARGET_COLS)
    tr_df = add_base_pred_to_target(tr_df, pred_cols)

In [ ]:
if BASE_PRED_DIR is not None:
    scores = calc_score(tr_df, pred_cols)
    display(scores)

In [ ]:
if BASE_PRED_DIR is not None:
    utils.plot_calibration_curve(tr_df, pred_cols, n_bins=40)

## oofを保存

In [ ]:
def create_submission_csv(preds: pl.DataFrame, filename: str = "submission.csv"):
    submission_cols = TARGET_COLS

    # validate preds columns
    if len(preds.columns) != len(submission_cols):
        raise ValueError(
            f"preds columns must be {len(submission_cols)}, but got {len(preds.columns)}"
        )

    preds.columns = submission_cols
    preds.write_csv(filename)
    print(f"Submission file is created: {filename}")


# 元の順番に戻して保存
create_submission_csv(tr_df.sort("origin_idx").select(pred_cols), "oof_preds.csv")

## Submission

In [ ]:
preds = utils.predict(
    models,
    ts_images,
    ts_df,
    feature.columns,
    scene_id_col="scene_id",
    scene_dsec_col="scene_dsec",
    pred_cols=pred_cols,
)
pred_cols = preds.columns
ts_df = pl.concat([ts_df, preds], how="horizontal")

preds

In [ ]:
if BASE_PRED_DIR is not None:
    # 差し引いていた基礎推定値を足して元のtarget, pred列に戻す
    ts_df = add_base_pred_to_target(ts_df, pred_cols)
    display(ts_df)

In [ ]:
# 元の順番に戻す
ts_df = ts_df.sort("origin_idx")

In [ ]:
create_submission_csv(ts_df.select(pred_cols), "submission.csv")